# Chapter 8. Higher-order functions

- Function types
- Higher-order functions (HOFs) and their use for structuring code
	- just functions that take in other functions as params, or returns another function 
- Inline functions 
	- remove the performance costs of lambdas
- Non-local returns and labels
- Anonymous functions (i.e. lambdas!)

## 8.1. DECLARING HIGHER-ORDER FUNCTIONS

What is a HOF?
- a function that takes another function as an argument or returns one
- that function can be represented by lambdas or function references

### 8.1.1 How to declare a function

- `() -> Unit = { ... }`
- `(Int, String) -> Unit = { x, y -> ... } `
- `(String, String) -> String?` <- can be nullable

In [1]:
// a regular function type
val sum: (Int, Int) -> Int = { x, y -> x + y }

// return types can be nullable
val maybeNull: (Int, Int) -> Int? = { x, y -> if (x + y < 0) null else x + y }

// if the function itself is nullable, you'll need parentheses around whole thing
var funOrNull: ((Int, Int) -> Int)? = null

// higher-order-function
fun printResult(x: Int, y: Int, fn: (Int, Int) -> Any?) = println(fn(x, y))

printResult(1, 2, sum) 
printResult(-4, -3, maybeNull)
printResult(2, 3) { x, y -> "anonymous lambda called with: $x, $y" }

3
null
anonymous lambda called with: 2, 3


### 8.1.2. Calling functions passed as arguments

- Same as regular functions

In [9]:
// filter for only strings that match a predicate
fun List<String>.filter(predicate: (String) -> Boolean): List<String> {
    val filteredList: MutableList<String> = mutableListOf()
    for (word in this) {
        if (predicate(word)) filteredList.add(word)
    }
    return filteredList.toList()
}

println(listOf("do", "cat", "four").filter { word -> word.length > 2  })

[cat, four]


### 8.1.3. Using function types from Java

Under the hood - a variable of function type is instance of class implementing a FunctionN interface where N = number of arguments:
- Function0<R>, Function1<P1, R> etc
- Each function has a single `invoke`. 


In [22]:
/* Kotlin: Given this function in Kotlin... */
fun logIt(fn: (Int) -> Int) { println(fn(42)) }

/**
* Java (pre-8) - pass in an anonymous function that overrides invoke
logIt(new Function<Integer, Integer>() {
    @Override
    public Integer invoke(Integer number) {
        return number + 1;
    }
}
**/

/**
Note that if your Kotlin function returns Unit, like (String) -> Unit
you need to return Unit.INSTANCE in Java, *not* void

List<String> strings = listOf("42")
CollectionsKt.forEach(strings, { s -> {
    println(s)
    return Unit.INSTANCE
}})
**/

### 8.1.4. Default and null values for parameters with function types

In [6]:
// Approach 1: use a default function for your function params

fun <T> Collection<T>.joinToString(
    separator:String = ", ",
    transform: (T) -> String = { it.toString().toUpperCase() }
): String {
    val result = StringBuilder()
    for ((idx, el) in this.withIndex()) {
        if (idx > 0) result.append(separator)
        result.append(transform(el))
    }
    return result.toString()
}

val letters = listOf("cats", "dogs")
println(letters.joinToString())

CATS, DOGS


In [8]:
// Approach 2: make your function param nullable

fun <T> Collection<T>.joinToString(
    separator:String = ", ",
    transform: ((T) -> String)? = null
): String {
    val result = StringBuilder()
    for ((idx, el) in this.withIndex()) {
        if (idx > 0) result.append(separator)
        result.append(transform?.invoke(el) ?: el.toString().toUpperCase())
    }
    return result.toString()
}

val letters = listOf("cats", "dogs")
println(letters.joinToString())

CATS, DOGS


### 8.1.5. Returning functions from functions

Not as common as function params, but cool for currying!


In [2]:
// simple curry example

// a function that returns another function that adds some base num
fun addWithBase(base: Int): (Int) -> Int { 
    return { anotherNum -> (base + anotherNum) }
}

val addOne = addWithBase(1) 
println(addOne(41))
println(addWithBase(2)(4))

42
6


In [3]:
// Note that in current Kotlin versions, don't need to nest these classes inside the sealed class
sealed class Block {
    data class TextBlock(val text: String) : Block()
    data class ImageBlock(val url: String) : Block()
}

// each block type has its own "is valid" predicate
fun <T : Block> getBlockValidator(clazz: Class<T>): (T) -> Boolean {
    when(clazz){
        Block.TextBlock::class.java -> return { 
            b: Block -> b is Block.TextBlock && b.text.isNotBlank()
        }
        Block.ImageBlock::class.java -> return { 
            b: Block -> b is Block.ImageBlock && b.url.endsWith(".jpg")
        }
        else -> return { b: Block -> false }
    }
}

val TEXT_BLOCK_VALIDATOR = getBlockValidator(Block.TextBlock::class.java)
val IMAGE_BLOCK_VALIDATOR = getBlockValidator(Block.ImageBlock::class.java)

println(TEXT_BLOCK_VALIDATOR(Block.TextBlock(""))) // false
println(TEXT_BLOCK_VALIDATOR(Block.TextBlock("coalesce"))) // true
println(IMAGE_BLOCK_VALIDATOR(Block.ImageBlock("shook"))) // false
println(IMAGE_BLOCK_VALIDATOR(Block.ImageBlock("shook.jpg"))) // true


false
true
false
true


### 8.1.6. Removing duplication through lambdas

Function types and lambda expressions => reusable code.
Strategy pattern - use a general function type as strategy, and pass different lambda expressions as different strategies

In [ ]:
enum class PostAction { LIKE, REBLOG, LIGHTBOX, NOTES_CLICK }
enum class NavigationState { DASHBOARD, SEARCH, BLOG, UNKNOWN }

data class PostEvent(
    val action: PostAction,
    val screen: NavigationState = NavigationState.UNKNOWN
)

fun List<PostEvent>.logPostActions(predicate: (PostEvent) -> Boolean) = 
    filter(predicate).map { "{it.action} on {it.screen}" }

val postEvents = listOf(
    PostEvent(PostAction.LIKE, NavigationState.DASHBOARD),
    PostEvent(PostAction.REBLOG, NavigationState.BLOG),
    PostEvent(PostAction.NOTES_CLICK),
    PostEvent(PostAction.LIGHTBOX),
    PostEvent(PostAction.REBLOG, NavigationState.SEARCH)
)
// filter for events on blog screen
val blogScreenEvents = postEvents.logPostActions {
    it.screen == NavigationState.BLOG
}
// filter for reblog events on any scren
val reblogEvents = postEvents.logPostActions {
    it.action == PostAction.REBLOG
}
println(unknownScreenEvents)
println(reblogEvents)

## 8.2. INLINE FUNCTIONS: REMOVING THE OVERHEAD OF LAMBDAS

Do lambdas present some performance problems?

Lambdas are normally compiled to anonymous classes. Which means:
 - every time you use a lambda expression, an extra class is created  
 - if it captures some variables, then a new object is created on every invocation

### 8.2.1. How inlining works

The body of the lambda expression that is passed as an argument is substituted directly into the resulting code.

In [3]:
class Lock {
    fun lock() = println("locking...")
    fun unlock() = println("unlocking...")
}

// original inlined function
inline fun <T> synchronized(lock: Lock, action: () -> T): T {
    lock.lock()
    try {
        return action()
    }
    finally {
        lock.unlock()
    }
}

// calling function in kotlin
fun foo(l: Lock) {
    println("=== Before inline sync call == ")
    
    synchronized(l) {
        println("ACTION")
    }
    
    println("=== After inline sync call == ")
}

// calling function compiles to this in java
fun compiledFoo(l: Lock) {
    println("=== Before inline sync call == ")

    /* inlined fn */
    l.lock()
    try {
        println("ACTION")
    } finally {
        l.unlock()
    }
    
    println("=== After inline sync call == ")
}

foo(Lock())
compiledFoo(Lock())

=== Before inline sync call == 
locking...
ACTION
unlocking...
=== After inline sync call == 
=== Before inline sync call == 
locking...
ACTION
unlocking...
=== After inline sync call == 


In [3]:
// if the lambda code isn't available at the site where 
// the inline function is called, it's not inlined

class LockOwner(val l: Lock) {
    fun runUnderLock(action: () -> Unit) {
        synchronized(l, action)
    }
    
    // compiles to this in java
    fun compiledRunUnderLock(action: () -> Unit) {
        l.lock()
        try {
            action() // not inlined because there's no lambda
        } finally {
            l.unlock()
        }
    }
}

LockOwner(Lock()).runUnderLock {
    println("hi i am a lock")
}

// If you have two uses of an inline function in different 
// locations with different lambdas, then every call site will be 
// inlined independently. 
// The code of the inline function will be copied to both locations 
// where you use it, with different lambdas substituted into it.

hi i am a lock


#### 8.2.2. Restrictions on inline functions

Not every function that uses lambdas can be inlined:
- generally ok if it's called directly, or passed as an argument to another inline function
- but if the param is stored somewhere for later use, the code of the lambda expression can't be inlined because there must be an object that contains this code
- compiler will complain “Illegal usage of inline-parameter.”


In [ ]:
fun <T, R> Sequence<T>.map(transform: (T) -> R): Sequence<R> {
    // `transform` is stored as a property in TransformingSequence
    // so it has to use the standard non-inline representation
    // (anonymous class implementing Function1 interface)
    return TransformingSequence(this, transform)
}